In [1]:
import pandas as pd
print("pandas version: {}". format(pd.__version__))

import numpy as np
print("numpy version: {}". format(np.__version__))

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit, train_test_split, cross_val_score
from sklearn.metrics import precision_recall_curve, roc_curve, confusion_matrix, roc_auc_score
print("sklearn version: {}". format(sklearn.__version__))

import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint
print("keras version: {}". format(keras.__version__))

import tensorflow as tf
print("tensorflow version: {}". format(tf.__version__))

import optuna
print("optuna version: {}". format(optuna.__version__))

import mlflow
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from mlflow.tracking import MlflowClient
print("mlflow version: {}". format(mlflow.__version__))

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import os
import warnings
warnings.simplefilter('ignore')

pandas version: 1.4.3
numpy version: 1.23.2
sklearn version: 1.0.2
keras version: 2.10.0
tensorflow version: 2.10.0


e:\programs\anaconda3\envs\keras_mlflow\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


optuna version: 3.0.1
mlflow version: 1.28.0


In [2]:
RANDOM_STATE=42
N_TRAILS=10 # run for x runs
TIMEOUT=600 # run for max 10 minutes (if the last run at 9 minutes runs for 1h, the active run is not killed)

In [3]:
def load_data():
    # load prepared training and test dataset
    df_train = pd.read_pickle('../03_dataCleaningPreparation/df_train_prepared.pkl')
    df_test = pd.read_pickle('../03_dataCleaningPreparation/df_test_prepared.pkl')

    # split the training and test dataset to the input features (x_train, x_test) and the survival class (y_train)
    y_train = df_train['Survived']
    x_train = df_train.drop(['Survived'], axis=1)
    x_test = df_test

    x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.3, stratify=y_train, random_state=RANDOM_STATE)

    return x_train, y_train, x_validate, y_validate, x_test

x_train, y_train, x_validate, y_validate, x_test = load_data()

In [4]:
def evaluate_model(x_train, y_train, y_validate, y_validate_pred, y_validate_scores):
    """
    evaluate the classification model with
    - classification report
    - precision-recall-curve
    - ROC curve
    """


    def plot_confusion_matrix(y_validate, y_validate_pred):
        group_names = ["True Neg", "False Pos", "False Neg", "True Pos"]
        group_counts = ["{0:0.0f}".format(value) for value in
                        confusion_matrix(y_validate, y_validate_pred).flatten()]
        group_percentages = ["{0:.2%}".format(value) for value in
                             confusion_matrix(y_validate, y_validate_pred).flatten()/np.sum(confusion_matrix(y_validate, y_validate_pred))]
        labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
                  zip(group_names,group_counts,group_percentages)]
        labels = np.asarray(labels).reshape(2,2)

        fig2, ax2 = plt.subplots()
        sns.heatmap(confusion_matrix(y_validate, y_validate_pred), annot=labels, fmt="", cmap='Blues')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        plt.close()


    def plot_precision_recall_vs_threshold(y_validate, y_scores):
        precisions, recalls, thresholds = precision_recall_curve(y_validate, y_scores)

        # convert to f score
        fscore = (2 * precisions * recalls) / (precisions + recalls)
        # locate the index of the largest f score
        ix = np.argmax(fscore)
   

        fig3, ax3 = plt.subplots()
        ax3.plot(thresholds, precisions[:-1], "b", label="Precision")
        ax3.plot(thresholds, recalls[:-1], "g", label="Recall")
        ax3.plot(thresholds, fscore[:-1], "r", label="F1 Score")
        ax3.axvline(x=thresholds[ix], color='red', linestyle='--')
        plt.axhline(y=precisions[ix], color='b', linestyle='--')
        plt.axhline(y=recalls[ix], color='g', linestyle='--')
        ax3.set_xlabel("Threshold")
        ax3.legend(loc="upper left")
        ax3.set_ylim([0,1])
        plt.close()

        fig4, ax4 = plt.subplots()
        ax4.plot(recalls, precisions, marker='.', label='Logistic')
        ax4.scatter(recalls[ix], precisions[ix], 200, marker='o', color='red', label='Best')
        ax4.set_xlabel('Recall')
        ax4.set_ylabel('Precision')
        plt.close()
        

    def plot_roc_curve(y_validate, y_scores):
        fpr, tpr, thresholds = roc_curve(y_validate, y_scores)

        roc_auc = round(roc_auc_score(y_validate, y_scores), 3)
        
        optimal_idx = np.argmax(tpr - fpr)

        fig5, ax5 = plt.subplots()
        ax5.plot(fpr, tpr, linewidth=2)
        ax5.plot([0,1], [0,1], 'k--')
        ax5.axis([0,1,0,1])
        ax5.scatter(fpr[optimal_idx], tpr[optimal_idx], 200, marker='o', color='red', label='Best')
        ax5.set_xlabel('False Positive Rate')
        ax5.set_ylabel('True Positive Rate')
        plt.close()


        

    plot_confusion_matrix(y_validate, y_validate_pred)
    plot_precision_recall_vs_threshold(y_validate, y_validate_scores)
    plot_roc_curve(y_validate, y_validate_scores)

In [5]:
model = Sequential()
model.add(keras.Input(shape=(x_train.shape[1],)))

model.add(Dense(units=5))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(1, activation='sigmoid'))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 5)                 5315      
                                                                 
 batch_normalization (BatchN  (None, 5)                20        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 5)                 0         
                                                                 
 dropout (Dropout)           (None, 5)                 0         
                                                                 
 dense_1 (Dense)             (None, 1)                 6         
                                                                 
Total params: 5,341
Trainable params: 5,331
Non-trainable params: 10
_____________________________________________________

In [6]:
adam = keras.optimizers.Adam(lr=0.01, beta_1=0.9, beta_2=0.999, amsgrad=False)

# compile the keras model
model.compile(
    loss='binary_crossentropy',
    optimizer=adam,
    metrics=['accuracy']
    )


# SETUP A EARLY STOPPING CALL and model check point API
earlystopping = keras.callbacks.EarlyStopping(
    monitor='accuracy',
    patience=10,
    verbose=1,
    mode='min'
    )


checkpointer = ModelCheckpoint(
    filepath='bestvalue',
    moniter='val_loss',
    verbose=0,
    save_best_only=True
    )


callback_list = [earlystopping]



# fit the keras model on the dataset
model.fit(
    x_train, y_train,
    epochs=150,
    batch_size=10,
    callbacks=callback_list,
    validation_data=(x_validate, y_validate)
)

# # predict the training outcome
# y_validate_pred = model.predict(x_validate)

# # predict probabilities
# y_validate_proba = model.predict_proba(x_validate)
# # keep probabilities for the positive outcome only
# y_validate_scores = y_validate_proba[:, 1]


y_validate_scores = model.predict(x_validate)
y_validate_pred = tf.greater(y_validate_proba, .5)



evaluate_model(x_train, y_train, y_validate, y_validate_pred, y_validate_scores)

# evaluate the keras model
_, accuracy = model.evaluate(x_validate, y_validate)


Epoch 1/150
63/63 [==============================] - 2s 12ms/step - loss: 0.6567 - accuracy: 0.6404 - val_loss: 0.5697 - val_accuracy: 0.6269
Epoch 2/150
63/63 [==============================] - 0s 7ms/step - loss: 0.5466 - accuracy: 0.7496 - val_loss: 0.5266 - val_accuracy: 0.7313
Epoch 3/150
63/63 [==============================] - 1s 9ms/step - loss: 0.4935 - accuracy: 0.7480 - val_loss: 0.4651 - val_accuracy: 0.8022
Epoch 4/150
63/63 [==============================] - 1s 8ms/step - loss: 0.5221 - accuracy: 0.7303 - val_loss: 0.4921 - val_accuracy: 0.7575
Epoch 5/150
63/63 [==============================] - 1s 9ms/step - loss: 0.4368 - accuracy: 0.7945 - val_loss: 0.4697 - val_accuracy: 0.8097
Epoch 6/150
63/63 [==============================] - 1s 8ms/step - loss: 0.4913 - accuracy: 0.7640 - val_loss: 0.7130 - val_accuracy: 0.7015
Epoch 7/150
63/63 [==============================] - 1s 8ms/step - loss: 0.4591 - accuracy: 0.7801 - val_loss: 0.4699 - val_accuracy: 0.8209
Epoch 8/150


IndexError: index 1 is out of bounds for axis 1 with size 1

In [7]:
y_validate_proba

array([[4.20552939e-01],
       [1.15667218e-02],
       [4.63405550e-02],
       [2.64706332e-02],
       [6.78260718e-03],
       [3.94007176e-01],
       [1.78864356e-02],
       [9.45235550e-01],
       [1.06360182e-01],
       [6.76813245e-01],
       [8.59167993e-01],
       [9.49639678e-01],
       [7.51744390e-01],
       [6.16411567e-01],
       [4.31875139e-01],
       [9.08600926e-01],
       [9.87926871e-02],
       [6.84507489e-01],
       [8.74283075e-01],
       [4.20512676e-01],
       [2.46625952e-02],
       [7.69546926e-02],
       [8.77355397e-01],
       [1.61711164e-02],
       [1.03598885e-01],
       [7.06438348e-02],
       [9.71818030e-01],
       [6.27891421e-01],
       [1.61748622e-02],
       [6.44863784e-01],
       [8.95845532e-01],
       [9.53530014e-01],
       [4.70447429e-02],
       [9.26855326e-01],
       [9.54566479e-01],
       [6.68814600e-01],
       [6.22212775e-02],
       [4.28337544e-01],
       [1.66665599e-01],
       [1.20585658e-01],


In [8]:
y_validate_pred = tf.greater(y_validate_proba, .5)

In [9]:
y_validate_pred

<tf.Tensor: shape=(268, 1), dtype=bool, numpy=
array([[False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [ True],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [ True],
       [ True],
       [False],
       [ True],
       [False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [False],
       [ True],
       [ True],
       [False],
       [F

In [10]:
model.evaluate(x_validate, y_validate)

9/9 [==============================] - 0s 7ms/step - loss: 0.4353 - accuracy: 0.8134


[0.43529534339904785, 0.8134328126907349]